In [1]:
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage,BaseMessage
from langgraph.graph import StateGraph,START,END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import InMemorySaver
from typing import List,Annotated,TypedDict
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
model=ChatGroq(model="llama-3.1-8b-instant")

In [4]:
class StateSchema(TypedDict):
    message:Annotated[list[BaseMessage],add_messages] # Persistance for memory

In [5]:

def chatbot(state:StateSchema)->StateSchema:
    prompt=state['message']
    response=model.invoke(prompt)

    return {
        'message':[response]
    }


In [6]:
graph=StateGraph(StateSchema)

In [7]:
graph.add_node('chatbot',chatbot)

graph.add_edge(START,'chatbot')
graph.add_edge('chatbot',END)

workflow=graph.compile(InMemorySaver()) # To save you stuffs in RAM (erases as soon as we close this program. Also this is required becasue after each state the memory is erased so that chatbot will not remember your inforamtion, esteanially it is storing your data in RAM or even database)

In [8]:
thread_id='1' # One interaction with the chatbot

while True:

    user_msg=input("Enter your message")
    print(f"Human:{user_msg}")

    if user_msg.strip().lower() in ['exit','quit']:
        break

    initial_state={
        'message':[HumanMessage(content=user_msg)]
    }
    config={'configurable':{'thread_id':thread_id}}
    result=workflow.invoke(initial_state,config=config)['message'][-1].content
    print(f"AI:{result}")

Human:Hi , My name is Arya
AI:Nice to meet you, Arya. Is there something I can help you with or would you like to chat?
Human:Wait what was my name
AI:Your name is Arya.
Human:tell me one country that is best for travelling
AI:One country that is often considered great for traveling is New Zealand. Known for its stunning natural beauty, diverse landscapes, and outdoor activities, New Zealand offers a wide range of options for travelers, from hiking and adventure sports to scenic drives and cultural experiences. It's a popular destination for nature lovers, photographers, and those seeking a unique and unforgettable travel experience.
Human:sure give me tourist places of that country
AI:New Zealand is a paradise for tourists, with a plethora of amazing destinations to explore. Here are some of the top tourist places to visit in New Zealand:

1. **Milford Sound**: A fjord in Fiordland National Park, known for its breathtaking scenery, glaciers, and wildlife, including penguins and seals.